In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd
import re
import selenium

In [2]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": ChromeDriverManager().install()}
    return Browser("chrome", **executable_path, headless=False)

In [102]:
browser = init_browser()
url = "https://www.akc.org/dog-breeds/"
browser.visit(url)

#THERE ARE 24 PAGES OF DOG BREEDS
#https://www.akc.org/dog-breeds/page/24/

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/jenniferdean/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


In [ ]:
# breeds_list_of_dicts = []

for p in range(11):
    p += 1
    #####GO TO NEW BREED PAGE#####
    url = f"https://www.akc.org/dog-breeds/page/{p}/"
    browser.visit(url)
    
    for i in range(12):
        html = browser.html
        soup = bs(html, "html.parser")
        
        #####GO TO BREED DETAIL PAGE#####
        browser.find_by_css('a[class="d-block relative"]')[i].click()
        time.sleep(1)
        
        html = browser.html
        soup = bs(html, "html.parser")
        
        
        #####COLLECT DOG BREED NAME#####
        dog_breed = soup.find("h1").get_text()
        dog_breed = dog_breed.replace('\n', '').strip(" ")
        
        #####COLLECT DOG ATTRIBUTES#####
        attribute_block = soup.find_all("span", class_="attribute-list__description")
        
        if len(attribute_block) == 6:
            temperment = soup.find("span", class_="attribute-list__description attribute-list__text attribute-list__text--lg mb4 bpm-mb5 pb0 d-block").get_text()
            akc_rank = soup.find_all("span", class_="attribute-list__description attribute-list__text")[0].get_text()
            height = soup.find_all("span", class_="attribute-list__description attribute-list__text")[1].get_text()
            weight = soup.find_all("span", class_="attribute-list__description attribute-list__text")[2].get_text()
            life_ex = soup.find_all("span", class_="attribute-list__description attribute-list__text")[3].get_text()
            group = soup.find_all("span", class_="attribute-list__description attribute-list__text")[4].get_text()
        elif len(attribute_block) == 5: 
            temperment = soup.find("span", class_="attribute-list__description attribute-list__text attribute-list__text--lg mb4 bpm-mb5 pb0 d-block").get_text()
            height = soup.find_all("span", class_="attribute-list__description attribute-list__text")[0].get_text()
            weight = soup.find_all("span", class_="attribute-list__description attribute-list__text")[1].get_text()
            life_ex = soup.find_all("span", class_="attribute-list__description attribute-list__text")[2].get_text()
            group = soup.find_all("span", class_="attribute-list__description attribute-list__text")[3].get_text()
            akc_rank = "NA"
        elif len(attribute_block) == 4:
            temperment = soup.find("span", class_="attribute-list__description attribute-list__text attribute-list__text--lg mb4 bpm-mb5 pb0 d-block").get_text()
            height = soup.find_all("span", class_="attribute-list__description attribute-list__text")[0].get_text()
            life_ex = soup.find_all("span", class_="attribute-list__description attribute-list__text")[1].get_text()
            group = soup.find_all("span", class_="attribute-list__description attribute-list__text")[2].get_text()
            akc_rank = "NA"
            weight = "NA"
        else:
            print(f"Attributes are missing for '{dog_breed}'")
            temperment = f"Attributes are missing for '{dog_breed}'"
            akc_rank = f"Attributes are missing for '{dog_breed}'"
            height = f"Attributes are missing for '{dog_breed}'"
            weight = f"Attributes are missing for '{dog_breed}'"
            life_ex = f"Attributes are missing for '{dog_breed}'"
            group = f"Attributes are missing for '{dog_breed}'"
            
        #####COLLECT DOG IMAGE#####    
        try:
            image_url = soup.find_all('img', class_="media-wrap__image lozad")[1]["data-src"]
        except:
            print(f"Image is missing for '{dog_breed}'")
            image_url = f"Image is missing for '{dog_breed}'"
            
        #####COLLECT DOG DESCRIPTION#####
        try:
            description = soup.find("div", class_="breed-hero__footer").get_text()
            description = description.replace('\n', '').strip(" ")
        except:
            print(f"Description is missing for '{dog_breed}'")
            description = f"Description is missing for '{dog_breed}'"
            
        #####COLLECT DOG CARE SCALES#####    
        try:
            brushing = soup.find_all("div", class_="bar-graph__section")[0]["style"].strip('width: ;')
            shedding = soup.find_all("div", class_="bar-graph__section")[1]["style"].strip('width: ;')
            energy = soup.find_all("div", class_="bar-graph__section")[2]["style"].strip('width: ;')
            trainability = soup.find_all("div", class_="bar-graph__section")[3]["style"].strip('width: ;')
            temperment_scale = soup.find_all("div", class_="bar-graph__section")[4]["style"].strip('width: ;')
        except IndexError:
            print(f"Missing 'Care' Scales for '{dog_breed}'")
            brushing = f"Missing 'Care' Scales for '{dog_breed}'"
            shedding = f"Missing 'Care' Scales for '{dog_breed}'"
            energy = f"Missing 'Care' Scales for '{dog_breed}'"
            trainability = f"Missing 'Care' Scales for '{dog_breed}'"
            temperment_scale = f"Missing 'Care' Scales for '{dog_breed}'"
        
        #####COLLECT DOG COLORS #####
        try:
            table = soup.find_all('table')[0]
        
            color_list = []
            final_color_list = []

            for row in table.find_all('td'):
                if 'Description' in row:
                    continue
                elif 'Standard Colors' in row:
                    continue
                elif 'Registration Code' in row:
                    continue
                elif "Check Mark For Standard Color" in row:
                    continue
                elif "\n" in row:
                    continue
                else:
                    color = row.get_text()
                    color_list.append(color)
                for i in color_list:
                    if "Check Mark For Standard Color" in i:
                        continue
                    if "\n" in i:
                        continue
                    else:
                        try:
                            if int(i) > 0:
                                continue
                        except:
                            final_color_list.append(i)
        except:
            print(f"No Colors for '{dog_breed}'")
        
        #####GO BACK TO BREED PAGE#####
        browser.back()
        
        #####APPEND TO THE DICTIONARY#####
        breeds_list_of_dicts.append({"breed_name" : dog_breed,
                         "temperment": temperment,
                         "image" : image_url,
                         "description" : description,
                         "akc_rank" : akc_rank,
                         "height" : height,
                         "weight" : weight,
                         "life_expectancy" : life_ex,
                         "brushing_scale" : brushing,
                         "shedding_scale" : shedding,
                         "energy_scale" : energy,
                         "trainability_scale" : trainability,
                         "temperment_scale" : temperment_scale,
                         "color_options" : final_color_list
                        })
        time.sleep(1)
        
browser.quit()

In [ ]:
#Missing 'Care' Scales for 'Irish Red and White Setter'
#Missing 'Care' Scales for 'Irish Terrier'
#Missing 'Care' Scales for 'Jagdterrier'
#Missing 'Care' Scales for 'Hokkaido'
#No Colors for 'Japanese Terrier'
#Missing 'Care' Scales for 'Norrbottenspets'
#Missing 'Care' Scales for 'Perro de Presa Canario'
#Missing 'Care' Scales for 'Poodle (Miniature)'
#Missing 'Care' Scales for 'Porcelaine'
#Missing 'Care' Scales for 'Portuguese Sheepdog'
#Missing 'Care' Scales for 'Pudelpointer'
#Missing 'Care' Scales for 'Pyrenean Mastiff'
#Missing 'Care' Scales for 'Rafeiro do Alentejo'
#Missing 'Care' Scales for 'Segugio Italiano'
#Missing 'Care' Scales for 'Shikoku'
#Missing 'Care' Scales for 'Slovensky Cuvac'
#Missing 'Care' Scales for 'Slovensky Kopov'
#Missing 'Care' Scales for 'Spanish Mastiff'
#Missing 'Care' Scales for 'Spinone Italiano'
#Missing 'Care' Scales for 'Tornjak'
#Missing 'Care' Scales for 'Transylvanian Hound'
#Missing 'Care' Scales for 'Treeing Tennessee Brindle'
#Missing 'Care' Scales for 'Welsh Springer Spaniel'

In [123]:
breeds_list_of_dicts

[{'breed_name': 'Hovawart',
  'temperment': 'Alert, Faithful, Intelligent',
  'image': 'https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/02174018/Hovawart.10.jpg',
  'description': 'The Hovawart is a medium-temperament working dog with versatile usage and has a very good nose. He is kind, has an even disposition and protective instinct, is self-confident and has the ability to take stress. His balanced body proportions and special devotion to his family make him an outstanding companion, watch, guard, tracking and rescue-dog.',
  'akc_rank': 'NA',
  'height': '23-28 inches',
  'weight': '65-90 pounds',
  'life_expectancy': '10-14 years',
  'brushing_scale': '20%',
  'shedding_scale': '60%',
  'energy_scale': '60%',
  'trainability_scale': '20%',
  'temperment_scale': '40%',
  'color_options': ['Black', 'Black', 'Black', 'Blonde', 'Black', 'Blonde']},
 {'breed_name': 'Ibizan Hound',
  'temperment': 'Family-Oriented, Even-Tempered, Polite',
  'image': 'https://s

In [127]:
len(breeds_list_of_dicts) #Missing 'Care' Scales for 'Hokkaido'

137

In [88]:
html = browser.html
soup = bs(html, "html.parser")

In [124]:
html = browser.html
soup = bs(html, "html.parser")

#####COLLECT DOG BREED NAME#####
dog_breed = soup.find("h1").get_text()
dog_breed = dog_breed.replace('\n', '').strip(" ")

#####COLLECT DOG ATTRIBUTES#####
attribute_block = soup.find_all("span", class_="attribute-list__description")

if len(attribute_block) == 6:
    temperment = soup.find("span", class_="attribute-list__description attribute-list__text attribute-list__text--lg mb4 bpm-mb5 pb0 d-block").get_text()
    akc_rank = soup.find_all("span", class_="attribute-list__description attribute-list__text")[0].get_text()
    height = soup.find_all("span", class_="attribute-list__description attribute-list__text")[1].get_text()
    weight = soup.find_all("span", class_="attribute-list__description attribute-list__text")[2].get_text()
    life_ex = soup.find_all("span", class_="attribute-list__description attribute-list__text")[3].get_text()
    group = soup.find_all("span", class_="attribute-list__description attribute-list__text")[4].get_text()
elif len(attribute_block) == 5: 
    temperment = soup.find("span", class_="attribute-list__description attribute-list__text attribute-list__text--lg mb4 bpm-mb5 pb0 d-block").get_text()
    height = soup.find_all("span", class_="attribute-list__description attribute-list__text")[0].get_text()
    weight = soup.find_all("span", class_="attribute-list__description attribute-list__text")[1].get_text()
    life_ex = soup.find_all("span", class_="attribute-list__description attribute-list__text")[2].get_text()
    group = soup.find_all("span", class_="attribute-list__description attribute-list__text")[3].get_text()
    akc_rank = "NA"
elif len(attribute_block) == 4:
    temperment = soup.find("span", class_="attribute-list__description attribute-list__text attribute-list__text--lg mb4 bpm-mb5 pb0 d-block").get_text()
    height = soup.find_all("span", class_="attribute-list__description attribute-list__text")[0].get_text()
    life_ex = soup.find_all("span", class_="attribute-list__description attribute-list__text")[1].get_text()
    group = soup.find_all("span", class_="attribute-list__description attribute-list__text")[2].get_text()
    akc_rank = "NA"
    weight = "NA"
else:
    print(f"Attributes are missing for '{dog_breed}'")
    temperment = f"Attributes are missing for '{dog_breed}'"
    akc_rank = f"Attributes are missing for '{dog_breed}'"
    height = f"Attributes are missing for '{dog_breed}'"
    weight = f"Attributes are missing for '{dog_breed}'"
    life_ex = f"Attributes are missing for '{dog_breed}'"
    group = f"Attributes are missing for '{dog_breed}'"

#####COLLECT DOG IMAGE#####    
try:
    image_url = soup.find_all('img', class_="media-wrap__image lozad")[1]["data-src"]
except:
    print(f"Image is missing for '{dog_breed}'")
    image_url = f"Image is missing for '{dog_breed}'"

#####COLLECT DOG DESCRIPTION#####
try:
    description = soup.find("div", class_="breed-hero__footer").get_text()
    description = description.replace('\n', '').strip(" ")
except:
    print(f"Description is missing for '{dog_breed}'")
    description = f"Description is missing for '{dog_breed}'"

#####COLLECT DOG CARE SCALES#####    
try:
    brushing = soup.find_all("div", class_="bar-graph__section")[0]["style"].strip('width: ;')
    shedding = soup.find_all("div", class_="bar-graph__section")[1]["style"].strip('width: ;')
    energy = soup.find_all("div", class_="bar-graph__section")[2]["style"].strip('width: ;')
    trainability = soup.find_all("div", class_="bar-graph__section")[3]["style"].strip('width: ;')
    temperment_scale = soup.find_all("div", class_="bar-graph__section")[4]["style"].strip('width: ;')
except IndexError:
    print(f"Missing 'Care' Scales for '{dog_breed}'")
    brushing = f"Missing 'Care' Scales for '{dog_breed}'"
    shedding = f"Missing 'Care' Scales for '{dog_breed}'"
    energy = f"Missing 'Care' Scales for '{dog_breed}'"
    trainability = f"Missing 'Care' Scales for '{dog_breed}'"
    temperment_scale = f"Missing 'Care' Scales for '{dog_breed}'"

#####COLLECT DOG COLORS #####
try:
    table = soup.find_all('table')[0]

    color_list = []
    final_color_list = []

    for row in table.find_all('td'):
        if 'Description' in row:
            continue
        elif 'Standard Colors' in row:
            continue
        elif 'Registration Code' in row:
            continue
        elif "Check Mark For Standard Color" in row:
            continue
        elif "\n" in row:
            continue
        else:
            color = row.get_text()
            color_list.append(color)
        for i in color_list:
            if "Check Mark For Standard Color" in i:
                continue
            if "\n" in i:
                continue
            else:
                try:
                    if int(i) > 0:
                        continue
                except:
                    final_color_list.append(i)
except:
    print(f"No Colors for '{dog_breed}'")

#####GO BACK TO BREED PAGE#####
browser.back()

#####APPEND TO THE DICTIONARY#####
breeds_list_of_dicts.append({"breed_name" : dog_breed,
                 "temperment": temperment,
                 "image" : image_url,
                 "description" : description,
                 "akc_rank" : akc_rank,
                 "height" : height,
                 "weight" : weight,
                 "life_expectancy" : life_ex,
                 "brushing_scale" : brushing,
                 "shedding_scale" : shedding,
                 "energy_scale" : energy,
                 "trainability_scale" : trainability,
                 "temperment_scale" : temperment_scale,
                 "color_options" : final_color_list
                })
time.sleep(1)

#browser.quit()